In [1]:
%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

env: ANYWIDGET_HMR=1


In [20]:
import numpy as np
import json
from pathlib import Path
data = json.loads(Path("/Users/mpiekenbrock/Downloads/miserables.json").read_text())

node_ids = np.unique([n["id"] for n in data["nodes"]])
src_ids = np.array([e["source"] for e in data["links"]])
tgt_ids = np.array([e["target"] for e in data["links"]])

src_ids = np.searchsorted(node_ids, src_ids)
tgt_ids = np.searchsorted(node_ids, tgt_ids)
node_ids = np.searchsorted(node_ids, node_ids)

edgelist = np.c_[src_ids, tgt_ids]

import traitlets
import anywidget
class Widget(anywidget.AnyWidget):
    _esm = "pixiplex/static/widget.js"
    _css = "pixiplex/static/widget.css"
    
    width = traitlets.Int(250).tag(sync=True)
    height = traitlets.Int(250).tag(sync=True)
    
    node_ids = traitlets.List([]).tag(sync=True)
    src_ids = traitlets.List([]).tag(sync=True)
    tgt_ids = traitlets.List([]).tag(sync=True)
    
    def __init__(self, node_ids: np.ndarray, edgelist: np.ndarray, width: int = 250, height: int = 250) -> None:
        super().__init__()
        self.width = width 
        self.height = height
        self.node_ids = list(node_ids)
        self.src_ids = list(edgelist[:,0])
        self.tgt_ids = list(edgelist[:,1])
    

widget = Widget(node_ids, edgelist)
widget

Widget(node_ids=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,…

In [24]:
widget.send({ "type" : "msg:init_nodes"})

In [23]:
widget.send({ "type" : "msg:custom", "foo":"bar" })

In [17]:
dir(widget)

['__annotations__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_active_widgets',
 '_add_notifiers',
 '_all_trait_default_generators',
 '_anywidget_commands',
 '_anywidget_id',
 '_call_widget_constructed',
 '_comm_changed',
 '_compare',
 '_control_comm',
 '_cross_validation_lock',
 '_css',
 '_default_keys',
 '_descriptors',
 '_dom_classes',
 '_esm',
 '_gen_repr_from_keys',
 '_get_embed_state',
 '_get_trait_default_generator',
 '_handle_control_comm_msg',
 '_handle_custom_msg',
 '_handle_msg',
 '_holding_sync',
 '_instance_inits',
 '_is_numpy',
 '_lock_property',
 '_log_default',
 '_model_i

In [ ]:
# %% Embed statically -- this works!
import ipywidgets.embed
ipywidgets.embed.embed_minimal_html("forcegraph_embed.html", views=[widget], drop_defaults=False)